## Importing libraries and files

In [1]:
import pandas as pd
from keras.models import load_model
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys

ann_arr = load_model('Arrival_time/Arrival_ANN_model_3.h5')
ann_end = load_model('End_time/End_ANN_model_8.h5')
ann_chg = load_model('Change_time/Change_ANN_model_3.h5')
#Section1
ann1 = load_model('Pressure_profiles_codes_section_1_regression/ANN_13.h5')
sc1 = pickle.load(open('Pressure_profiles_codes_section_1_regression/scaler_ANN.pkl', 'rb'))
#Section2
ann2 = load_model('Pressure_profiles_codes_section_2_regression/ANN_1.h5')
sc2 = pickle.load(open('Pressure_profiles_codes_section_2_regression/scaler_ANN.pkl', 'rb'))
#Section3
ann3 = load_model('Pressure_profiles_codes_section_3_regression/ANN_1.h5')
sc3 = pickle.load(open('Pressure_profiles_codes_section_3_regression/scaler_ANN.pkl', 'rb'))
#Section4
ann4 = load_model('Pressure_profiles_codes_section_4_regression/ANN_1.h5')
sc4 = pickle.load(open('Pressure_profiles_codes_section_4_regression/scaler_ANN.pkl', 'rb'))
#Section5
ann5 = load_model('Pressure_profiles_codes_section_5_regression/ANN_1.h5')
sc5 = pickle.load(open('Pressure_profiles_codes_section_5_regression/scaler_ANN.pkl', 'rb'))

## Input details

In [62]:
charge_mass = 20
time = 0.005
atm_pressure = 101200
domain_min = -10
domain_max = 10
mesh_size = 400
if np.sqrt(domain_min**2 + domain_max**2) > 30:
    print ("out of trained range")
    #sys.exit()

In [63]:
x_vals = np.linspace(domain_min, domain_max, mesh_size)
y_vals = np.linspace(domain_min, domain_max, mesh_size)
X, Y = np.meshgrid(x_vals, y_vals)
#generate standoff distances
Z = np.sqrt(X**2 + Y**2)
#reshape Z
c_d = Z.reshape(-1)
c_d = c_d.reshape(len (c_d), 1)
c_ud = np.unique(c_d)
print ('c_d shape', c_d.shape, 'c_ud shape', c_ud.shape)

c_d shape (160000, 1) c_ud shape (31325,)


In [64]:
d_s0 = np.array([])
d_s1 = np.array([])
d_s2 = np.array([])
d_s3 = np.array([])
d_s4 = np.array([])
d_s5 = np.array([])
print (d_s0, d_s1, d_s2, d_s3, d_s4, d_s5)
for d in c_ud:
    if 3 <= d <= 7:
        d_s1 = np.append(d_s1,d)
    elif 7 < d <= 13:
        d_s2 = np.append(d_s2,d)
    elif 13 < d <= 19:
        d_s3 = np.append(d_s3,d) 
    elif 19 < d <= 25:
        d_s4 = np.append(d_s4,d)
    elif 25 < d <= 30:
        d_s5 = np.append(d_s5,d)
    else:
        d_s0 = np.append(d_s0,d)
print ('0', d_s0.shape, '1', d_s1.shape, '2', d_s2.shape, '3',d_s3.shape, '4', d_s4.shape, '5', d_s5.shape)
d_s0 = d_s0.reshape(len (d_s0), 1)
d_s1 = d_s1.reshape(len (d_s1), 1)
d_s2 = d_s2.reshape(len (d_s2), 1)
d_s3 = d_s3.reshape(len (d_s3), 1)
print ('0', d_s0.shape, '1', d_s1.shape, '2', d_s2.shape, '3',d_s3.shape, '4', d_s4.shape, '5', d_s5.shape)

[] [] [] [] [] []
0 (3229,) 1 (10756,) 2 (16813,) 3 (527,) 4 (0,) 5 (0,)
0 (3229, 1) 1 (10756, 1) 2 (16813, 1) 3 (527, 1) 4 (0,) 5 (0,)


## d_s0

In [65]:
c_p0 = np.repeat(100000, len(d_s0)).reshape(len(d_s0),1)
c_p0 = c_p0.flatten()
print (c_p0.shape)

(3229,)


## d_s1

In [66]:
#Charge mass array
c_m1 = np.repeat(charge_mass, len(d_s1)).reshape(len(d_s1),1)
print ('c_m1', c_m1.shape)
#Time array
c_t1 = np.repeat(time, len(d_s1)).reshape(len(d_s1),1)
print ('c_t1', c_t1.shape)
c_arr1 = np.concatenate ((c_m1, d_s1), axis = 1)
#predict change, arrival and end times
change_time1 = ann_chg.predict(c_arr1)
#print (change_time1)
arrival_time1 = ann_arr.predict(c_arr1)
end_time1 = ann_end.predict(c_arr1)

#define phase change
if time < change_time1[0]:
    c_ph1 = np.array([[0, 1]])
else:
    c_ph1 = np.array([[1, 0]])
    
positive = np.array([0, 1])
negative = np.array([1, 0])
for c_time in change_time1[1:]:
    #print (c_time)
    if time < c_time:
        c_ph1=np.vstack((c_ph1,positive))
    else:
        c_ph1=np.vstack((c_ph1,negative))
        
print ('d_s1', d_s1.shape)
print ('c_ph1', c_ph1.shape)
c_arr1_2 = np.concatenate ((c_m1, d_s1, c_t1, c_ph1), axis = 1)
c_arr1_2[:,0:3] = sc1.transform(c_arr1_2[:,0:3])
c_p1 = ann1.predict(c_arr1_2)
c_p1 = c_p1.flatten()
print (c_p1.shape)

c_m1 (10756, 1)
c_t1 (10756, 1)
337/337 [==============================] - 0s 1ms/step
d_s1 (10756, 1)
c_ph1 (10756, 2)
337/337 [==============================] - 1s 2ms/step
(10756,)


## d_s2

In [67]:
#Charge mass array
c_m2 = np.repeat(charge_mass, len(d_s2)).reshape(len(d_s2),1)
#Time array
c_t2 = np.repeat(time, len(d_s2)).reshape(len(d_s2),1)
c_arr2 = np.concatenate ((c_m2, d_s2), axis = 1)
#predict change, arrival and end times
change_time2 = ann_chg.predict(c_arr2)
arrival_time2 = ann_arr.predict(c_arr2)
end_time2 = ann_end.predict(c_arr2)

#define phase change
if time < change_time2[0]:
    c_ph2 = np.array([[0, 1]])
else:
    c_ph2 = np.array([[1, 0]])
    
positive = np.array([0, 1])
negative = np.array([1, 0])
for c_time in change_time2[1:]:
    if time < c_time:
        c_ph2=np.vstack((c_ph2,positive))
    else:
        c_ph2=np.vstack((c_ph2,negative))

c_arr2_2 = np.concatenate ((c_m2, d_s2, c_t2, c_ph2), axis = 1)
c_arr2_2[:,0:3] = sc2.transform(c_arr2_2[:,0:3])
c_p2 = ann2.predict(c_arr2_2)
c_p2 = c_p2.flatten()
print (c_p2.shape)

526/526 [==============================] - 1s 2ms/step
(16813,)


## d_s3

In [68]:
#Charge mass array
c_m3 = np.repeat(charge_mass, len(d_s3)).reshape(len(d_s3),1)
#Time array
c_t3 = np.repeat(time, len(d_s3)).reshape(len(d_s3),1)
c_arr3 = np.concatenate ((c_m3, d_s3), axis = 1)
#predict change, arrival and end times
change_time3 = ann_chg.predict(c_arr3)
arrival_time3 = ann_arr.predict(c_arr3)
end_time3 = ann_end.predict(c_arr3)

#define phase change
if time < change_time3[0]:
    c_ph3 = np.array([[0, 1]])
else:
    c_ph3 = np.array([[1, 0]])
    
positive = np.array([0, 1])
negative = np.array([1, 0])
for c_time in change_time3[1:]:
    if time < c_time:
        c_ph3=np.vstack((c_ph3,positive))
    else:
        c_ph3=np.vstack((c_ph3,negative))

c_arr3_2 = np.concatenate ((c_m3, d_s3, c_t3, c_ph3), axis = 1)
c_arr3_2[:,0:3] = sc3.transform(c_arr3_2[:,0:3])
c_p3 = ann3.predict(c_arr3_2)
c_p3 = c_p3.flatten()
print (c_p3.shape)

17/17 [==============================] - 0s 2ms/step
(527,)


## d_s4

In [30]:
#Charge mass array
c_m4 = np.repeat(charge_mass, len(d_s4)).reshape(len(d_s4),1)
#Time array
c_t4 = np.repeat(time, len(d_s4)).reshape(len(d_s4),1)
c_arr4 = np.concatenate ((c_m4, d_s4), axis = 1)
#predict change, arrival and end times
change_time4 = ann_chg.predict(c_arr4)
arrival_time4 = ann_arr.predict(c_arr4)
end_time4 = ann_end.predict(c_arr4)

#define phase change
if time < change_time4[0]:
    c_ph4 = np.array([[0, 1]])
else:
    c_ph4 = np.array([[1, 0]])
    
positive = np.array([0, 1])
negative = np.array([1, 0])
for c_time in change_time3[1:]:
    if time < c_time:
        c_ph4=np.vstack((c_ph4,positive))
    else:
        c_ph4=np.vstack((c_ph4,negative))

c_arr4_2 = np.concatenate ((c_m4, d_s4, c_t4, c_ph4), axis = 1)
c_arr4_2[:,0:3] = sc4.transform(c_arr4_2[:,0:3])
c_p4 = ann4.predict(c_arr4_2)
c_p4 = c_p4.flatten()
print (c_p4.shape)

382/382 [==============================] - 1s 1ms/step
(12218,)


## d_s5

In [30]:
#Charge mass array
c_m5 = np.repeat(charge_mass, len(d_s5)).reshape(len(d_s5),1)
#Time array
c_t5 = np.repeat(time, len(d_s5)).reshape(len(d_s5),1)
c_arr5 = np.concatenate ((c_m5, d_s5), axis = 1)
#predict change, arrival and end times
change_time5 = ann_chg.predict(c_arr5)
arrival_time5 = ann_arr.predict(c_arr5)
end_time5 = ann_end.predict(c_arr5)

#define phase change
if time < change_time5[0]:
    c_ph5 = np.array([[0, 1]])
else:
    c_ph5 = np.array([[1, 0]])
    
positive = np.array([0, 1])
negative = np.array([1, 0])
for c_time in change_time3[1:]:
    if time < c_time:
        c_ph5=np.vstack((c_ph5,positive))
    else:
        c_ph5=np.vstack((c_ph5,negative))

c_arr5_2 = np.concatenate ((c_m5, d_s5, c_t5, c_ph5), axis = 1)
c_arr5_2[:,0:3] = sc5.transform(c_arr5_2[:,0:3])
c_p5 = ann5.predict(c_arr5_2)
c_p5 = c_p5.flatten()
print (c_p5.shape)

382/382 [==============================] - 1s 1ms/step
(12218,)


In [69]:
d_s0 = d_s0.flatten()
d_s1 = d_s1.flatten()
d_s2 = d_s2.flatten()
d_s3 = d_s3.flatten()
#d_s4 = d_s4.flatten()
#d_s5 = d_s5.flatten()
print (d_s0.shape, d_s1.shape, d_s2.shape, d_s3.shape)
value_map = dict(zip(d_s0, c_p0))
value_map.update(dict(zip(d_s1, c_p1)))
value_map.update(dict(zip(d_s2, c_p2)))
value_map.update(dict(zip(d_s3, c_p3)))
#value_map.update(dict(zip(d_s4, c_p4)))
#value_map.update(dict(zip(d_s5, c_p5)))
print (len(value_map))
original_replaced = np.vectorize(value_map.get)(c_d)
print("Replaced array:", original_replaced)

(3229,) (10756,) (16813,) (527,)
31325
Replaced array: [[265084.78]
 [264648.4 ]
 [264213.12]
 ...
 [264213.12]
 [264648.4 ]
 [265084.78]]


In [36]:
print (original_replaced.shape)

(160000, 1)


In [70]:
print (time)

0.005


In [22]:
for index, (c_pressure,  c_arrival, c_end) in enumerate(zip(c_p, arrival_time, end_time)):
    if time < c_arrival:
        #print (c_arrival)
        c_p[index] = atm_pressure
        
    elif time > c_end:
        print (c_end)
        c_p[index] = atm_pressure

NameError: name 'arrival_time' is not defined

In [71]:
c_p = original_replaced.reshape(400, 400)
print (c_p)

[[265084.78 264648.4  264213.12 ... 264213.12 264648.4  265084.78]
 [264648.4  264210.9  263774.6  ... 263774.6  264210.9  264648.4 ]
 [264213.12 263774.6  263337.1  ... 263337.1  263774.6  264213.12]
 ...
 [264213.12 263774.6  263337.1  ... 263337.1  263774.6  264213.12]
 [264648.4  264210.9  263774.6  ... 263774.6  264210.9  264648.4 ]
 [265084.78 264648.4  264213.12 ... 264213.12 264648.4  265084.78]]


In [72]:
# convert array into dataframe
DF = pd.DataFrame(c_p)
 
# save the dataframe as a csv file
DF.to_csv("data1.csv")

PermissionError: [Errno 13] Permission denied: 'data1.csv'

In [1]:
import matplotlib
from scipy.interpolate import griddata

cdict = {'red': ((0.0, 0.0, 0.0),
                 (0.25, 0.2, 0.2),
                 (0.5, 0.0, 0.0),
                 (0.75, 1.0, 1.0),
                 (1.0, 1.0, 1.0)),
        'green':((0.0, 0.0, 0.0),
                 (0.25, 1.0, 1.0),
                 (0.5, 1.0, 1.0),
                 (0.75, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
        'blue': ((0.0, 1.0, 1.0),
                 (0.25, 1.0, 1.0),
                 (0.5, 0.0, 0.0),
                 (0.75, 0.0, 0.0),
                 (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)

#rng = np.random.default_rng()
#points = rng.random((1000, 2))

#c_p_interpolate = griddata(points, c_p, (X,Y), method = 'linear')

fig = plt.figure(figsize=(6,5))
left, bottom, width, height = 0.1, 0.1, 0.8, 0.8
ax = fig.add_axes([left, bottom, width, height])
levels = np.linspace(1000, 692000, 101)
cp = plt.contourf(X, Y, c_p, levels= levels, cmap = my_cmap) # c_p_interpolate
plt.colorbar(cp)

ax.set_title('Incident Pressure Contour Plot')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
plt.show()

NameError: name 'plt' is not defined